<a href="https://colab.research.google.com/github/youngsikwon/Py_ML_Deep_Learning/blob/master/KF94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

>  코로나 바이러스 때문에, 마스크 대란이 일어났다...<br>
kf94 마스크를 판매하는 사이트를 크롤링하여 텔레그램 봇을 통해 notify를 구현했다.<br>
kf94 재고가 있으면 notify 하게끔 구현했다...



# 라이브러리
 - selenium
 - telegram
 - yaml

 # 현재 크롤링 하는 사이트
  - 웰킵스
  - 생활공작소
  - 위케어

# 추가할 사이트
 - 농협몰
 - 우체국몰
 - 쿠팡
 - 티몬
 - 위메프
 - 네이버 쇼핑
 - 등등..

In [15]:
# 코랩 selenium 설치 방법
##!apt-get update
##apt install chromium-chromedirver
##!apt /usr/lib/chromium-browser/chromedriver/usr/bin
#!pip install selenium
##from selenium import webdriver


  Created wheel for telegram: filename=telegram-0.0.1-cp36-none-any.whl size=1308 sha256=a45e402e94dc2837cf59a97cf4e6c6d865a929e498edf5273f1c56bb558b5684
  Stored in directory: /root/.cache/pip/wheels/21/a4/d2/c289fe2e3a3050eefb01b2f1f9ef42346272a21bc3082a4342
Successfully built telegram


In [0]:
from selenium import webdriver
import yaml
import telegram

# yaml 파일 load
with open('config.yml') as f:
    conf = yaml.load(f,  Loader=yaml.FullLoader)

# 텔레그램 설정
bot = telegram.Bot(token = conf['telegram_token'])

# updates = bot.getUpdates()
# for u in updates:
#     print(u.message)

# 크롬 드라이버 설정
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")
# 혹은 options.add_argument("--disable-gpu")

driver = webdriver.Chrome('chromedriver', options=options)

# 웰킵스 크롤릴
def welkeepsmall_crawling():
    # 사이트 접속
    driver.get('http://www.welkeepsmall.com/shop/shopbrand.html?type=X&xcode=023')
    # 상품 리스트 가져오기
    contents = driver.find_elements_by_css_selector('div.tb-center')
    for content in contents:
        # 품절 확인
        soldout = content.find_element_by_css_selector('li.soldout')
        if soldout is None or soldout.text == 'SOLD OUT':
            continue
        # 품절이 아닌 상품 리스트
        item_title = content.find_element_by_css_selector('li.dsc').text
        bot.sendMessage(chat_id=conf['telegram_chat_id'], text="웰킵스: " + item_title)

def naver_store_crawling(market, url):
    # 사이트 접속
    driver.get(url)
    # 상품 리스트 가져오기
    items = driver.find_elements_by_xpath("//div[@class='module_list_product_listview']/ul[@class='list']/li[@class='item']")
    for item in items:
        # 품절인지 확인
        soldout = driver.find_elements_by_xpath("//div[@class='area_status']/div[@class='label soldouot']")
        if soldout is not None:
            continue
        # 상품 제목 element
        content = item.find_element_by_css_selector('strong.title')
        if content is None:
            continue
        # 상품 제목 프린트
        item_title = content.text
        if "kf94" in item_title.strip().lower():
            bot.sendMessage(chat_id=conf['telegram_chat_id'], text=market + ": " + item_title)

while True:
    welkeepsmall_crawling()
    # 생활 공작소
    naver_store_crawling('생활공작소','https://smartstore.naver.com/livingcrafts/search?q=%EB%A7%88%EC%8A%A4%ED%81%AC')
    # 위케어
    naver_store_crawling('위케어', 'https://smartstore.naver.com/wiicare/search?q=%EB%A7%88%EC%8A%A4%ED%81%AC')
    # DDOS 인식 막기 위해 일정 주기 추가
    driver.implicitly_wait(3)

driver.quit()